In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd /global/homes/p/pr4santh/Projects/SYMBA_SSM/

/global/u1/p/pr4santh/Projects/SYMBA_SSM


In [4]:
import numpy as np

In [5]:
import torch
import torch.nn as nn
from collections import defaultdict
from typing import Dict, Optional, Union

def count_parameters(model: nn.Module,
                    trainable_only: bool = False,
                    detailed: bool = False) -> Union[int, Dict[str, int]]:
    """
    Count the number of parameters in a PyTorch model and its submodules.

    Args:
        model: PyTorch model (nn.Module)
        trainable_only: If True, count only trainable parameters (default: False)
        detailed: If True, return detailed breakdown by submodule (default: False)

    Returns:
        int: Total number of parameters (if detailed=False)
        Dict[str, int]: Parameter counts by submodule (if detailed=True)
    """

    if detailed:
        param_counts = defaultdict(int)

        # Count parameters for each named module
        for name, module in model.named_modules():
            module_name = name if name else "root"

            for param in module.parameters(recurse=False):  # Don't double count
                if not trainable_only or param.requires_grad:
                    param_counts[module_name] += param.numel()

        # Convert to regular dict and sort by parameter count
        param_counts = dict(sorted(param_counts.items(),
                                 key=lambda x: x[1], reverse=True))
        return param_counts

    else:
        # Simple total count
        if trainable_only:
            return sum(p.numel() for p in model.parameters() if p.requires_grad)
        else:
            return sum(p.numel() for p in model.parameters())


def print_parameter_summary(model: nn.Module,
                          trainable_only: bool = False,
                          show_details: bool = True):
    """
    Print a formatted summary of model parameters.

    Args:
        model: PyTorch model
        trainable_only: If True, count only trainable parameters
        show_details: If True, show breakdown by submodule
    """

    total_params = count_parameters(model, trainable_only=trainable_only)
    param_type = "Trainable" if trainable_only else "Total"

    print(f"\n{'='*50}")
    print(f"MODEL PARAMETER SUMMARY")
    print(f"{'='*50}")
    print(f"{param_type} Parameters: {total_params:,}")
    print(f"Memory Usage: ~{total_params * 4 / 1024**2:.2f} MB (float32)")

    if show_details:
        detailed_counts = count_parameters(model, trainable_only=trainable_only, detailed=True)

        print(f"\nParameter Breakdown by Module:")
        print(f"{'-'*50}")
        print(f"{'Module Name':<30} {'Parameters':<15} {'%'}")
        print(f"{'-'*50}")

        for module_name, param_count in detailed_counts.items():
            if param_count > 0:  # Only show modules with parameters
                percentage = (param_count / total_params) * 100
                print(f"{module_name:<30} {param_count:<15,} {percentage:>5.1f}%")


# Enc Dec

In [ ]:
# cd /global/homes/p/pr4santh/Projects/SYMBA_SSM/model

/global/u1/p/pr4santh/Projects/SYMBA_SSM/model


In [17]:
!python mamba_encdec.py

Traceback (most recent call last):
  File "/global/u1/p/pr4santh/Projects/SYMBA_SSM/model/mamba_encdec.py", line 15, in <module>
    from .helpers.flash_cross_attention import FlashCrossAttentionWrapper
ImportError: attempted relative import with no known parent package


In [6]:
from model.mamba_encdec import MambaEncDec

/global/homes/p/pr4santh/miniconda3/envs/ssm_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
num_encoder_layers = 3
num_decoder_layers = 4

In [8]:
encoder_layers = ['Mamba'] * 2 * (num_encoder_layers - 1) + ['FMHSA', 'FFN']
decoder_layers = ['FXA', 'FFN'] + ['Mamba'] * 2 * (num_decoder_layers - 2) + ['FXA', 'FFN']
# decoder_layers = ['Mamba'] * 2 * (num_decoder_layers - 1)


# encoder_layers = ['FMHSA', 'FFN'] * (config.num_encoder_layers)
# decoder_layers = ['FXA', 'FFN'] * (config.num_decoder_layers)

mamba_config = {
    "enc_n_layer": (num_encoder_layers - 1) * 2,
    "d_model": 512,
    "dec_n_layer": (num_decoder_layers - 2) * 2,
    "rms_norm": True,
    "fused_add_norm": True,
    "use_fast_path": False,
    "encoder_layer_list": encoder_layers,
    "decoder_layer_list": decoder_layers,
    # "learning_rate": config.learning_rate,
    # "warmup_steps": config.warmup_steps,
    # "weight_decay": config.weight_decay,
    # "devices": config.devices
}
# mamba_config = {
#     "enc_n_layer": config.num_encoder_layers,
#     "d_model": config.d_model,
#     "n_layer": config.n_layer,
#     "rms_norm": config.rms_norm,
#     "fused_add_norm": config.fused_add_norm,
#     "use_fast_path": config.use_fast_path,
#     "learning_rate": config.learning_rate,
#     "warmup_steps": config.warmup_steps,
#     "weight_decay": config.weight_decay,
#     "devices": config.devices
# }

model = MambaEncDec(
    **mamba_config,
    config = mamba_config,
    src_vocab_size=412,
    tgt_vocab_size=39
)


In [9]:
model

MambaEncDec(
  (encoder): MixerModel(
    (embedding): Embedding(412, 512)
    (layers): ModuleList(
      (0-3): 4 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=512, out_features=2048, bias=False)
          (conv1d): Conv1d(1024, 1024, kernel_size=(4,), stride=(1,), padding=(3,), groups=1024)
          (act): SiLU()
          (x_proj): Linear(in_features=1024, out_features=64, bias=False)
          (dt_proj): Linear(in_features=32, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=512, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (decoder): MambaDecoder(
    (backbone): MixerModel(
      (embedding): Embedding(39, 512)
      (layers): ModuleList(
        (0): FlashCrossAttentionWrapper(
          (attention): Attention(
            (to_q): Linear(in_features=512, out_features=512, bias=False)
            (to_k): Linear(in_features=512, out_features=512, bias=False)
       

In [10]:
print_parameter_summary(model)


MODEL PARAMETER SUMMARY
Total Parameters: 14,603,520
Memory Usage: ~55.71 MB (float32)

Parameter Breakdown by Module:
--------------------------------------------------
Module Name                    Parameters      %
--------------------------------------------------
decoder.backbone.layers.1.mlp.fc1 2,101,248        14.4%
decoder.backbone.layers.1.mlp.fc2 1,049,088         7.2%
encoder.layers.0.mixer.in_proj 1,048,576         7.2%
encoder.layers.1.mixer.in_proj 1,048,576         7.2%
encoder.layers.2.mixer.in_proj 1,048,576         7.2%
encoder.layers.3.mixer.in_proj 1,048,576         7.2%
decoder.backbone.layers.2.mixer.in_proj 1,048,576         7.2%
decoder.backbone.layers.3.mixer.in_proj 1,048,576         7.2%
encoder.layers.0.mixer.out_proj 524,288           3.6%
encoder.layers.1.mixer.out_proj 524,288           3.6%
encoder.layers.2.mixer.out_proj 524,288           3.6%
encoder.layers.3.mixer.out_proj 524,288           3.6%
decoder.backbone.layers.2.mixer.out_proj 524,288     

In [9]:
model

MambaEncDec(
  (encoder): MixerModel(
    (embedding): Embedding(412, 512)
    (layers): ModuleList(
      (0-3): 4 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=512, out_features=2048, bias=False)
          (conv1d): Conv1d(1024, 1024, kernel_size=(4,), stride=(1,), padding=(3,), groups=1024)
          (act): SiLU()
          (x_proj): Linear(in_features=1024, out_features=64, bias=False)
          (dt_proj): Linear(in_features=32, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=512, bias=False)
        )
        (norm): RMSNorm()
      )
      (4): FlashSelfAttentionWrapper(
        (rotary_pos_emb): RotaryEmbedding()
        (attention): Attention(
          (to_q): Linear(in_features=512, out_features=512, bias=False)
          (to_k): Linear(in_features=512, out_features=512, bias=False)
          (to_v): Linear(in_features=512, out_features=512, bias=False)
          (split_q_heads): Rearrange('b n (h d) -> b h 

In [10]:
encoder_layers = ['FMHSA', 'FFN'] * (num_encoder_layers)
decoder_layers = ['FXA', 'FFN'] * (num_decoder_layers)


# encoder_layers = ['FMHSA', 'FFN'] * (config.num_encoder_layers)
# decoder_layers = ['FXA', 'FFN'] * (config.num_decoder_layers)

mamba_config = {
    "enc_n_layer": num_encoder_layers * 2,
    "d_model": 512,
    "dec_n_layer": num_decoder_layers * 2,
    "rms_norm": True,
    "fused_add_norm": True,
    "use_fast_path": False,
    "encoder_layer_list": encoder_layers,
    "decoder_layer_list": decoder_layers,
    # "learning_rate": config.learning_rate,
    # "warmup_steps": config.warmup_steps,
    # "weight_decay": config.weight_decay,
    # "devices": config.devices
}
# mamba_config = {
#     "enc_n_layer": config.num_encoder_layers,
#     "d_model": config.d_model,
#     "n_layer": config.n_layer,
#     "rms_norm": config.rms_norm,
#     "fused_add_norm": config.fused_add_norm,
#     "use_fast_path": config.use_fast_path,
#     "learning_rate": config.learning_rate,
#     "warmup_steps": config.warmup_steps,
#     "weight_decay": config.weight_decay,
#     "devices": config.devices
# }

model = MambaEncDec(
    **mamba_config,
    config = mamba_config,
    src_vocab_size=412,
    tgt_vocab_size=39
)


In [11]:
print_parameter_summary(model)


MODEL PARAMETER SUMMARY
Total Parameters: 29,632,512
Memory Usage: ~113.04 MB (float32)

Parameter Breakdown by Module:
--------------------------------------------------
Module Name                    Parameters      %
--------------------------------------------------
encoder.layers.1.mlp.fc1       2,101,248         7.1%
encoder.layers.3.mlp.fc1       2,101,248         7.1%
encoder.layers.5.mlp.fc1       2,101,248         7.1%
decoder.backbone.layers.1.mlp.fc1 2,101,248         7.1%
decoder.backbone.layers.3.mlp.fc1 2,101,248         7.1%
decoder.backbone.layers.5.mlp.fc1 2,101,248         7.1%
decoder.backbone.layers.7.mlp.fc1 2,101,248         7.1%
encoder.layers.1.mlp.fc2       1,049,088         3.5%
encoder.layers.3.mlp.fc2       1,049,088         3.5%
encoder.layers.5.mlp.fc2       1,049,088         3.5%
decoder.backbone.layers.1.mlp.fc2 1,049,088         3.5%
decoder.backbone.layers.3.mlp.fc2 1,049,088         3.5%
decoder.backbone.layers.5.mlp.fc2 1,049,088         3.5%
decod

# Dec only

In [6]:
!python model/mamba_dec.py

In [4]:
from model.mamba_dec import MambaMT

/global/homes/p/pr4santh/miniconda3/envs/ssm_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
mamba_config = {
    "d_model": 512,
    "n_layer": 8,
    "rms_norm": True,
    "fused_add_norm": True,
    "pad_vocab_size_multiple": 2
    # "use_fast_path": False,
}

model = MambaMT(
    **mamba_config,
    config = mamba_config,
    src_vocab_size=412,
    tgt_vocab_size=48,
    precision="bf16-mixed"
    # tgt_vocab_size=39,
)

In [13]:
model = model.to(device='cuda:0')

In [5]:
model

MambaMT(
  (model): MambaLMHeadModel(
    (backbone): MixerModel(
      (embedding): Embedding(416, 512)
      (layers): ModuleList(
        (0-7): 8 x Block(
          (norm): RMSNorm()
          (mixer): Mamba(
            (in_proj): Linear(in_features=512, out_features=2048, bias=False)
            (conv1d): Conv1d(1024, 1024, kernel_size=(4,), stride=(1,), padding=(3,), groups=1024)
            (act): SiLU()
            (x_proj): Linear(in_features=1024, out_features=64, bias=False)
            (dt_proj): Linear(in_features=32, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=512, bias=False)
          )
        )
      )
      (norm_f): RMSNorm()
    )
    (lm_head): Linear(in_features=512, out_features=416, bias=False)
  )
  (lm_head): Linear(in_features=512, out_features=48, bias=False)
)

In [7]:
src = np.random.randint(low=0, high=418, size=5)
tgt = np.random.randint(low=0, high=49, size=5)

src_padding_mask = np.random.randint(low=0, high=2, size=len(src))
# Generate a causal target mask
# This is a lower triangular matrix of 1s, preventing the model from attending to future tokens.
tgt_padding_mask = np.tril(np.ones((len(tgt), len(tgt)), dtype=int))

In [9]:
src_padding_mask

array([1, 0, 0, 1, 1])

In [22]:
%%time
out = model.model.forward(input_ids = torch.tensor(src,device='cuda:0').unsqueeze(0), position_ids=None, inference_params=None)

CPU times: user 1.39 s, sys: 322 ms, total: 1.71 s
Wall time: 2.18 s


In [26]:
out2 = model.forward(input_ids = torch.tensor(src,device='cuda:0').unsqueeze(0), position_ids=None, inference_params=None)

In [28]:
out.logits.shape, out2.logits.shape

(torch.Size([1, 5, 416]), torch.Size([1, 5, 48]))

# Transformer

In [2]:
cd /global/homes/p/pr4santh/Projects/SineKAN/

/global/u1/p/pr4santh/Projects/SineKAN


In [4]:
from model.model import Model

In [6]:
t_model = Model(
    3, #config.num_encoder_layers,
    4, #config.num_decoder_layers,
    512, #config.embedding_size,
    8, #config.nhead,
    149, #config.src_voc_size,
    94, #config.tgt_voc_size,
    512, #config.ff_dims,
    0.1, #config.dropout,
    False, #config.is_pre_norm,
    False, # config.is_kan,
    device='cuda',# config.kan_ff_dims,
    # config.kan_grid_size,
    #config.device
)

In [9]:
print_parameter_summary(t_model)


MODEL PARAMETER SUMMARY
Total Parameters: 15,425,118
Memory Usage: ~58.84 MB (float32)

Parameter Breakdown by Module:
--------------------------------------------------
Module Name                    Parameters      %
--------------------------------------------------
transformer.encoder.layers.0.self_attn.q_proj 262,656           1.7%
transformer.encoder.layers.0.self_attn.k_proj 262,656           1.7%
transformer.encoder.layers.0.self_attn.v_proj 262,656           1.7%
transformer.encoder.layers.0.self_attn.out_proj 262,656           1.7%
transformer.encoder.layers.0.linear1 262,656           1.7%
transformer.encoder.layers.0.linear2 262,656           1.7%
transformer.encoder.layers.1.self_attn.q_proj 262,656           1.7%
transformer.encoder.layers.1.self_attn.k_proj 262,656           1.7%
transformer.encoder.layers.1.self_attn.v_proj 262,656           1.7%
transformer.encoder.layers.1.self_attn.out_proj 262,656           1.7%
transformer.encoder.layers.1.linear1 262,656         

# misc

In [5]:
model = model.to(device='cuda:0')

In [7]:
model.to('cpu')

MambaEncDec(
  (encoder): MixerModel(
    (embedding): Embedding(300, 512)
    (layers): ModuleList(
      (0-3): 4 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=512, out_features=2048, bias=False)
          (conv1d): Conv1d(1024, 1024, kernel_size=(4,), stride=(1,), padding=(3,), groups=1024)
          (act): SiLU()
          (x_proj): Linear(in_features=1024, out_features=64, bias=False)
          (dt_proj): Linear(in_features=32, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=512, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (decoder): MambaDecoder(
    (backbone): MixerModel(
      (embedding): Embedding(51, 512)
      (layers): ModuleList(
        (0): Block(
          (mixer): Mamba(
            (in_proj): Linear(in_features=512, out_features=2048, bias=False)
            (conv1d): Conv1d(1024, 1024, kernel_size=(4,), stride=(1,), padding=(3,), groups=1024)
       

In [8]:
model = model.to('cpu')

In [9]:
del model
import torch
torch.cuda.empty_cache()